## Exercises in KMeans clustering with Spark and Scala

Based on material from Advanced Analytics with Spark: Ryza, Laserson, Owen & Wills Chapter 5

Uses Almond Scala kernel for Jupyter notebooks. 
Scala version 2.12

Data set is the KDD Cup 1999 Data Set available at http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
The full dataset (c. 740MB was used)

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`
import org.apache.spark.sql._

Downloaded https://repo1.maven.org/maven2/sh/almond/almond-spark_2.12/0.8.2/almond-spark_2.12-0.8.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/2.4.3/spark-sql_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.12/2.4.3/spark-parent_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/18/apache-18.pom
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-column/1.10.1/parquet-column-1.10.1.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/ammonite-spark_2.12/0.7.2/ammonite-spark_2.12-0.7.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-tags_2.12/2.4.3/spark-tags_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-catalyst_2.12/2.4.3/spark-catalyst_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-databind/2.6.7.1/jackson-databind-2.6.7.1.pom
Downloaded https://repo1.maven.org/maven2

Downloaded https://repo1.maven.org/maven2/net/sf/py4j/py4j/0.10.7/py4j-0.10.7.pom
Downloaded https://repo1.maven.org/maven2/log4j/log4j/1.2.17/log4j-1.2.17.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-network-shuffle_2.12/2.4.3/spark-network-shuffle_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/containers/jersey-container-servlet/2.22.2/jersey-container-servlet-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/com/ning/compress-lzf/1.0.3/compress-lzf-1.0.3.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-kvstore_2.12/2.4.3/spark-kvstore_2.12-2.4.3.pom
Downloaded https://repo1.maven.org/maven2/org/lz4/lz4-java/1.4.0/lz4-java-1.4.0.pom
Downloaded https://repo1.maven.org/maven2/com/google/protobuf/protobuf-java/2.5.0/protobuf-java-2.5.0.pom
Downloaded https://repo1.maven.org/maven2/org/eclipse/jetty/jetty-server/9.4.20.v20190813/jetty-server-9.4.20.v20190813.pom
Downloaded https://repo1.maven.org/maven2/oro/oro

Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/core/jersey-server/2.22.2/jersey-server-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/com/twitter/chill_2.12/0.9.3/chill_2.12-0.9.3.pom
Downloaded https://repo1.maven.org/maven2/org/apache/arrow/arrow-format/0.10.0/arrow-format-0.10.0.pom
Downloaded https://repo1.maven.org/maven2/org/xerial/snappy/snappy-java/1.1.7.3/snappy-java-1.1.7.3.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.25/slf4j-api-1.7.25.pom
Downloaded https://repo1.maven.org/maven2/commons-codec/commons-codec/1.10/commons-codec-1.10.pom
Downloaded https://repo1.maven.org/maven2/com/vlkan/flatbuffers/1.2.0-3f79e055/flatbuffers-1.2.0-3f79e055.pom
Downloaded https://repo1.maven.org/maven2/net/razorvine/pyrolite/4.13/pyrolite-4.13.pom
Downloaded https://repo1.maven.org/maven2/org/codehaus/janino/commons-compiler/3.0.9/commons-compiler-3.0.9.pom
Downloaded https://repo1.maven.org/maven2/com/clearspring/analytics/stream/2.7.0/stream-2

Downloaded https://repo1.maven.org/maven2/org/eclipse/jetty/jetty-project/9.4.20.v20190813/jetty-project-9.4.20.v20190813.pom
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/paranamer/paranamer-parent/2.8/paranamer-parent-2.8.pom
Downloaded https://repo1.maven.org/maven2/org/apache/apache/7/apache-7.pom
Downloaded https://repo1.maven.org/maven2/org/antlr/antlr4-master/4.7/antlr4-master-4.7.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-parent/2.7/jackson-parent-2.7.pom
Downloaded https://repo1.maven.org/maven2/org/roaringbitmap/RoaringBitmapParent/0.7.45/RoaringBitmapParent-0.7.45.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/jackson-parent/2.6.1/jackson-parent-2.6.1.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-parent/40/commons-parent-40.pom
Downloaded https://repo1.maven.org/maven2/org/slf4j/slf4j-parent/1.7.16/slf4j-parent-1.7.16.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/

Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/module/jackson-module-paranamer/2.7.9/jackson-module-paranamer-2.7.9.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/bundles/repackaged/jersey-guava/2.22.2/jersey-guava-2.22.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-app/2.6.5/hadoop-mapreduce-client-app-2.6.5.pom
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-common/2.6.5/hadoop-common-2.6.5.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-reflect/2.12.8/scala-reflect-2.12.8.pom
Downloaded https://repo1.maven.org/maven2/com/fasterxml/jackson/core/jackson-annotations/2.6.7/jackson-annotations-2.6.7.pom
Downloaded https://repo1.maven.org/maven2/org/fusesource/leveldbjni/leveldbjni-all/1.8/leveldbjni-all-1.8.pom
Downloaded https://repo1.maven.org/maven2/org/glassfish/hk2/external/javax.inject/2.4.0-b34/javax.inject-2.4.0-b34.pom
Downloaded https://repo1.maven.org/maven2/or

...

Downloaded https://repo1.maven.org/maven2/javax/inject/javax.inject/1/javax.inject-1-sources.jar
Downloaded https://repo1.maven.org/maven2/com/google/inject/guice/3.0/guice-3.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-yarn-server-common/2.6.5/hadoop-yarn-server-common-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/fusesource/leveldbjni/leveldbjni-all/1.8/leveldbjni-all-1.8-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-hdfs/2.6.5/hadoop-hdfs-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-core/2.6.5/hadoop-mapreduce-client-core-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-yarn-client/2.6.5/hadoop-yarn-client-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-yarn-common/2.6.5/hadoop-yarn-common-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/codehaus/jettison

Downloaded https://repo1.maven.org/maven2/org/glassfish/jersey/bundles/repackaged/jersey-guava/2.22.2/jersey-guava-2.22.2-sources.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/hk2/external/aopalliance-repackaged/2.4.0-b34/aopalliance-repackaged-2.4.0-b34-sources.jar
Downloaded https://repo1.maven.org/maven2/org/json4s/json4s-core_2.12/3.5.3/json4s-core_2.12-3.5.3-sources.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/hk2/external/javax.inject/2.4.0-b34/javax.inject-2.4.0-b34-sources.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/hk2/hk2-api/2.4.0-b34/hk2-api-2.4.0-b34-sources.jar
Downloaded https://repo1.maven.org/maven2/org/javassist/javassist/3.18.1-GA/javassist-3.18.1-GA-sources.jar
Downloaded https://repo1.maven.org/maven2/org/glassfish/hk2/hk2-locator/2.4.0-b34/hk2-locator-2.4.0-b34-sources.jar
Downloaded https://repo1.maven.org/maven2/org/json4s/json4s-ast_2.12/3.5.3/json4s-ast_2.12-3.5.3-sources.jar
Downloaded https://repo1.maven.org/maven

Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-shims/1.5.5/orc-shims-1.5.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-encoding/1.10.1/parquet-encoding-1.10.1-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/orc/orc-mapreduce/1.5.5/orc-mapreduce-1.5.5-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-jackson/1.10.1/parquet-jackson-1.10.1-sources.jar
Downloaded https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-graphite/3.1.5/metrics-graphite-3.1.5.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-catalyst_2.12/2.4.3/spark-catalyst_2.12-2.4.3-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-common/1.10.1/parquet-common-1.10.1-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-encoding/1.10.1/parquet-encoding-1.10.1.jar
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-format/2.4.0/p

Downloaded https://repo1.maven.org/maven2/org/apache/directory/api/api-util/1.0.0-M20/api-util-1.0.0-M20-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-mapreduce-client-common/2.6.5/hadoop-mapreduce-client-common-2.6.5-sources.jar
Downloaded https://repo1.maven.org/maven2/xmlenc/xmlenc/0.52/xmlenc-0.52-sources.jar.sha1
Downloaded https://repo1.maven.org/maven2/xerces/xercesImpl/2.9.1/xercesImpl-2.9.1-sources.jar.sha1
Downloaded https://repo1.maven.org/maven2/io/netty/netty-all/4.1.17.Final/netty-all-4.1.17.Final-sources.jar


import $ivy.$                                  

import org.apache.spark.sql._

In [2]:
import org.apache.log4j.{Logger,Level}
Logger.getLogger("org").setLevel(Level.ERROR)

import org.apache.log4j.{Logger,Level}


In [3]:
val spark = {
    org.apache.spark.sql.NotebookSparkSession.builder().master("local[*]").config("spark.sql.shuffle.partitions","8").getOrCreate()
}

Loading spark-stubs


Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.7.2/spark-stubs_24_2.12-0.7.2.pom
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.7.2/spark-stubs_24_2.12-0.7.2-sources.jar
Downloaded https://repo1.maven.org/maven2/sh/almond/spark-stubs_24_2.12/0.7.2/spark-stubs_24_2.12-0.7.2.jar


Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@2b240e5e

In [4]:
val dataNoHeader = spark.read
            .option("inferSchema","true")
            .option("header","false")
            .csv("../KDDCup1999_dataset/kddcup.data")

csv at cmd3.sc:4

1 / 1

csv at cmd3.sc:4

8 / 8

dataNoHeader: DataFrame = [_c0: int, _c1: string ... 40 more fields]

In [5]:
dataNoHeader.columns.length

res4: Int = 42

Get the list of headers from the kddcup.names file and use to convert to a DF

In [6]:
val names = spark.read.option("header","false").csv("../KDDCup1999_dataset/kddcup.names")

csv at cmd5.sc:1

1 / 1

names: DataFrame = [_c0: string, _c1: string ... 21 more fields]

In [7]:
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._

In [8]:
names.createOrReplaceTempView("temp1")

In [9]:
import org.apache.spark.sql.types.DataTypes._

import org.apache.spark.sql.types.DataTypes._

Trim the text after the : to get just the names

In [10]:
val nameDF = spark.sql("select left(_c0, instr(_c0,':')-1) as text from temp1 where instr(_c0,':')>0 union all select 'label'")
val nameArray = nameDF.collect.map(row=>row.getString(0))
nameArray.length

collect at cmd9.sc:2

2 / 2

nameDF: DataFrame = [text: string]
nameArray: Array[String] = Array(
  "duration",
  "protocol_type",
  "service",
  "flag",
  "src_bytes",
  "dst_bytes",
  "land",
  "wrong_fragment",
  "urgent",
  "hot",
  "num_failed_logins",
  "logged_in",
  "num_compromised",
  "root_shell",
  "su_attempted",
  "num_root",
  "num_file_creations",
  "num_shells",
  "num_access_files",
  "num_outbound_cmds",
  "is_host_login",
  "is_guest_login",
  "count",
  "srv_count",
  "serror_rate",
  "srv_serror_rate",
  "rerror_rate",
  "srv_rerror_rate",
  "same_srv_rate",
  "diff_srv_rate",
  "srv_diff_host_rate",
  "dst_host_count",
  "dst_host_srv_count",
  "dst_host_same_srv_rate",
  "dst_host_diff_srv_rate",
  "dst_host_same_src_port_rate",
  "dst_host_srv_diff_host_rate",
  "dst_host_serror_rate",
...
res9_2: Int = 42

In [11]:
val data = dataNoHeader.toDF(nameArray:_*) // _* converts array to a varags (variable number of arguments)

data: DataFrame = [duration: int, protocol_type: string ... 40 more fields]

In [12]:
data.select("label").groupBy("label").count().orderBy(col("count").desc).show

show at cmd11.sc:1

8 / 8

show at cmd11.sc:1

8 / 8

+----------------+-------+
|           label|  count|
+----------------+-------+
|          smurf.|2807886|
|        neptune.|1072017|
|         normal.| 972781|
|          satan.|  15892|
|        ipsweep.|  12481|
|      portsweep.|  10413|
|           nmap.|   2316|
|           back.|   2203|
|    warezclient.|   1020|
|       teardrop.|    979|
|            pod.|    264|
|   guess_passwd.|     53|
|buffer_overflow.|     30|
|           land.|     21|
|    warezmaster.|     20|
|           imap.|     12|
|        rootkit.|     10|
|     loadmodule.|      9|
|      ftp_write.|      8|
|       multihop.|      7|
+----------------+-------+
only showing top 20 rows



Create pipeline for the kmeans and run int

In [13]:
import $ivy.`org.apache.spark::spark-mllib:2.4.0`

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-mllib_2.12/2.4.0/spark-mllib_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-parent_2.12/2.4.0/spark-parent_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze_2.12/0.13.2/breeze_2.12-0.13.2.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-tags_2.12/2.4.0/spark-tags_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-mllib-local_2.12/2.4.0/spark-mllib-local_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-sql_2.12/2.4.0/spark-sql_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-graphx_2.12/2.4.0/spark-graphx_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-streaming_2.12/2.4.0/spark-streaming_2.12-2.4.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-core_2.12/2.4.0/spark-core_2.12-2.4.0.pom
Downloaded http

Downloaded https://repo1.maven.org/maven2/com/thoughtworks/paranamer/paranamer-parent/2.7/paranamer-parent-2.7.pom
Downloaded https://repo1.maven.org/maven2/org/codehaus/codehaus-parent/4/codehaus-parent-4.pom
Downloaded https://repo1.maven.org/maven2/commons-beanutils/commons-beanutils-core/1.8.0/commons-beanutils-core-1.8.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-tags_2.12/2.4.0/spark-tags_2.12-2.4.0.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-unsafe_2.12/2.4.0/spark-unsafe_2.12-2.4.0.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-launcher_2.12/2.4.0/spark-launcher_2.12-2.4.0.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-kvstore_2.12/2.4.0/spark-kvstore_2.12-2.4.0.jar
Downloaded https://repo1.maven.org/maven2/commons-beanutils/commons-beanutils-core/1.8.0/commons-beanutils-core-1.8.0.jar
Downloaded https://repo1.maven.org/maven2/org/roaringbitmap/RoaringBitmap/0.5.11/RoaringBitmap-0.5.1

Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-network-common_2.12/2.4.0/spark-network-common_2.12-2.4.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-network-shuffle_2.12/2.4.0/spark-network-shuffle_2.12-2.4.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/spire-math/spire_2.12/0.13.0/spire_2.12-0.13.0.jar
Downloaded https://repo1.maven.org/maven2/net/sf/opencsv/opencsv/2.3/opencsv-2.3.jar
Downloaded https://repo1.maven.org/maven2/org/xerial/snappy/snappy-java/1.1.7.1/snappy-java-1.1.7.1-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/parquet/parquet-hadoop/1.10.0/parquet-hadoop-1.10.0-sources.jar
Downloaded https://repo1.maven.org/maven2/com/github/fommil/netlib/core/1.1.2/core-1.1.2-sources.jar
Downloaded https://repo1.maven.org/maven2/org/scalanlp/breeze_2.12/0.13.2/breeze_2.12-0.13.2-sources.jar
Downloaded https://repo1.maven.org/maven2/org/apache/spark/spark-mllib_2.12/2.4.0/spark-mllib_2.12-2.4.0.jar
Do

import $ivy.$                                    

In [14]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.clustering.{KMeans, KMeansModel}
import org.apache.spark.ml.feature.VectorAssembler

import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.clustering.{KMeans, KMeansModel}

import org.apache.spark.ml.feature.VectorAssembler

In [15]:
data.getClass

res14: Class[T] = class org.apache.spark.sql.Dataset

In [16]:
data.schema

res15: types.StructType = StructType(
  StructField("duration", IntegerType, true, {}),
  StructField("protocol_type", StringType, true, {}),
  StructField("service", StringType, true, {}),
  StructField("flag", StringType, true, {}),
  StructField("src_bytes", IntegerType, true, {}),
  StructField("dst_bytes", IntegerType, true, {}),
  StructField("land", IntegerType, true, {}),
  StructField("wrong_fragment", IntegerType, true, {}),
  StructField("urgent", IntegerType, true, {}),
  StructField("hot", IntegerType, true, {}),
  StructField("num_failed_logins", IntegerType, true, {}),
  StructField("logged_in", IntegerType, true, {}),
  StructField("num_compromised", IntegerType, true, {}),
  StructField("root_shell", IntegerType, true, {}),
  StructField("su_attempted", IntegerType, true, {}),
  StructField("num_root", IntegerType, true, {}),
  StructField("num_file_creations", IntegerType, true, {}),
  StructField("num_shells", IntegerType, true, {}),
  StructField("num_access_files",

In [17]:
val numericalCols = data.drop("protocol_type","service","flag").cache

numericalCols: Dataset[Row] = [duration: int, src_bytes: int ... 37 more fields]

In [18]:
val assembler = new VectorAssembler().
                setInputCols(numericalCols.columns.filter(_ != "label")).
                setOutputCol("featureVector")

assembler: VectorAssembler = vecAssembler_64ab2a841c8a

Create null model with default options

In [19]:
val kmeans = new KMeans().
    setPredictionCol("cluster").
    setFeaturesCol("featureVector")

kmeans: KMeans = kmeans_06d72513f4b2

Create pipeline for transforming ( assembel it into the input object expected of all ML alg), fit the model. This sets it up for hyper parameter tuning later using the spark.ml.tuning api

In [20]:
val pipeline = new Pipeline().setStages(Array(assembler,kmeans))
val pipelineModel = pipeline.fit(numericalCols)

takeSample at KMeans.scala:386

8 / 8

takeSample at KMeans.scala:386

8 / 8

sum at KMeans.scala:406

...

collect at ClusteringSummary.scala:51

8 / 8

collect at ClusteringSummary.scala:51

8 / 8

pipeline: Pipeline = pipeline_778832daf5bb
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_778832daf5bb

In [21]:
val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
kmeansModel.clusterCenters.foreach(println)

[48.34019491959669,1834.6215497618625,826.2031900016945,5.7161172049003456E-6,6.487793027561892E-4,7.961734678254053E-6,0.012437658596734055,3.205108575604837E-5,0.14352904910348827,0.00808830584493399,6.818511237273984E-5,3.6746467745787934E-5,0.012934960793560386,0.0011887482315762398,7.430952366370449E-5,0.0010211435092468404,0.0,4.082940860643104E-7,8.351655530445469E-4,334.9735084506668,295.26714620807076,0.17797031701994256,0.1780369894027269,0.05766489875327379,0.05772990937912744,0.7898841322630906,0.02117961060991097,0.028260810096297884,232.98107822302248,189.21428335201279,0.7537133898007772,0.03071097882384052,0.605051930924901,0.006464107887636894,0.1780911843182284,0.1778858981346887,0.05792761150001272,0.05765922142401037]
[10999.0,0.0,1.309937401E9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,255.0,1.0,0.0,0.65,1.0,0.0,0.0,0.0,1.0,1.0]


kmeansModel: KMeansModel = kmeans_06d72513f4b2

In [22]:
val withCluster = pipelineModel.transform(numericalCols)

withCluster: DataFrame = [duration: int, src_bytes: int ... 39 more fields]

In [23]:
withCluster("cluster")

res22: Column = cluster

In [24]:
import spark.implicits._

import spark.implicits._

In [25]:
withCluster.select("cluster","label").groupBy("cluster","label").count()
                .orderBy($"cluster",$"count".desc).show(25)

show at cmd24.sc:2

8 / 8

show at cmd24.sc:2

8 / 8

+-------+----------------+-------+
|cluster|           label|  count|
+-------+----------------+-------+
|      0|          smurf.|2807886|
|      0|        neptune.|1072017|
|      0|         normal.| 972781|
|      0|          satan.|  15892|
|      0|        ipsweep.|  12481|
|      0|      portsweep.|  10412|
|      0|           nmap.|   2316|
|      0|           back.|   2203|
|      0|    warezclient.|   1020|
|      0|       teardrop.|    979|
|      0|            pod.|    264|
|      0|   guess_passwd.|     53|
|      0|buffer_overflow.|     30|
|      0|           land.|     21|
|      0|    warezmaster.|     20|
|      0|           imap.|     12|
|      0|        rootkit.|     10|
|      0|     loadmodule.|      9|
|      0|      ftp_write.|      8|
|      0|       multihop.|      7|
|      0|            phf.|      4|
|      0|           perl.|      3|
|      0|            spy.|      2|
|      1|      portsweep.|      1|
+-------+----------------+-------+



In [26]:
kmeansModel.extractParamMap

res25: org.apache.spark.ml.param.ParamMap = {
	kmeans_06d72513f4b2-distanceMeasure: euclidean,
	kmeans_06d72513f4b2-featuresCol: featureVector,
	kmeans_06d72513f4b2-initMode: k-means||,
	kmeans_06d72513f4b2-initSteps: 2,
	kmeans_06d72513f4b2-k: 2,
	kmeans_06d72513f4b2-maxIter: 20,
	kmeans_06d72513f4b2-predictionCol: cluster,
	kmeans_06d72513f4b2-seed: -1689246527,
	kmeans_06d72513f4b2-tol: 1.0E-4
}

Note that the default number of clusters is 2.

Because this is unsupervised there isn't an obvious loss function that can be used to choose optimal parameters such as in the case of regression where purity or entropy could be used. The loss function to be used in the sum of squared distance to the closest centres which intuitively should be minimised. However, this is not implemented in org.apache.spark.evaluators api and must be done manually

In [27]:
import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.DataFrame

In [28]:
import scala.util.Random

import scala.util.Random

def myClusterScore(data: DataFrame, k: Int): Double = {
    val assembler = new VectorAssembler().setInputCols(data.columns.filter(_!="label")).setOutputCol("featureVector")
    val kmeans = new KMeans().setFeaturesCol("featureVector").setSeed(Random.nextLong()).setK(k).setPredictionCol("cluster")
    
    val pipeline= new Pipeline().setStages(Array(assembler,kmeans))
    val kmeansModel=pipeline.fit(data).stages.last.asInstanceOf[KMeansModel]
    kmeansModel.computeCost(assembler.transform(data))/data.count()
}

(20 to 100 by 20).map(k=>myClusterScore(numericalCols,k)).foreach(println)

Do a cross tab on the predicitons of the Kmeans model to see the 'purity' of the clusters

kmeansModel.extractParamMap

In [29]:
val preds = kmeansModel.transform(assembler.transform(numericalCols))

preds: DataFrame = [duration: int, src_bytes: int ... 39 more fields]

In [30]:
val labels = data.select("label").as[String].collect

collect at cmd29.sc:1

8 / 8

labels: Array[String] = Array(
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
  "normal.",
...

In [31]:
val predCol = preds.select("cluster").as[Int].collect()

collect at cmd30.sc:1

8 / 8

predCol: Array[Int] = Array(
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
...

This code cell below seems to be problematic. It leads to GC overflow. Trying other ways wiht less GC overhead.

val xtab = spark.createDataset(labels.zip(predCol))

In [32]:
val xtab = labels.zip(predCol)

xtab: Array[(String, Int)] = Array(
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
  ("normal.", 0),
...

In [33]:
val xtabLabeled= spark.createDataset(xtab)

xtabLabeled: Dataset[(String, Int)] = [_1: string, _2: int]

In [34]:
xtabLabeled.getClass

res33: Class[T] = class org.apache.spark.sql.Dataset

In [35]:
xtabLabeled.cache()

res34: Dataset[(String, Int)] = [_1: string, _2: int]

xtabLabeled.groupBy("_2").pivot("_1").count().show()

The above code took a very long time to run ( groupby must be a pretty in efficient??)

Add some categorical variables to the clustering

In [29]:
import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}

import org.apache.spark.ml.feature.{OneHotEncoder, StringIndexer}

In [30]:
def oneHotPipeline(inputCol: String):(Pipeline, String) = {
    val indexer = new StringIndexer().
        setInputCol(inputCol).
        setOutputCol(inputCol+"_indexed")
    val encoder = new OneHotEncoder().
        setInputCol(inputCol+"_indexed").
        setOutputCol(inputCol+"_vec")
    val pipeline = new Pipeline().setStages(Array(indexer,encoder))
    return (pipeline,inputCol+"_vec")
}

defined function oneHotPipeline

Create the vector asembler to deal with the new onehot encoded data

In [31]:
val (protoEncoder,protoCol) = oneHotPipeline("protocol_type")

val (serviceEncoder, serviceCol) = oneHotPipeline("service")

val (flagEncoder, flagCol) = oneHotPipeline("flag")

protoEncoder: Pipeline = pipeline_1d7bde83709e
protoCol: String = "protocol_type_vec"
serviceEncoder: Pipeline = pipeline_5bd4a7470e4a
serviceCol: String = "service_vec"
flagEncoder: Pipeline = pipeline_b8a49daa2f96
flagCol: String = "flag_vec"

Put the new one-hot columns in to replace the categorical columns

In [32]:
import org.apache.spark.ml.feature.StandardScaler

import org.apache.spark.ml.feature.StandardScaler

In [33]:
val assembledCols = Set(data.columns: _*) --
            Seq("label","protocol_type","service","flag") ++
            Seq(protoCol, serviceCol,flagCol)

assembledCols: Set[String] = Set(
  "num_file_creations",
  "root_shell",
  "protocol_type_vec",
  "duration",
  "dst_host_same_srv_rate",
  "dst_host_srv_count",
  "count",
  "dst_host_srv_rerror_rate",
  "land",
  "urgent",
  "serror_rate",
  "src_bytes",
  "logged_in",
  "num_failed_logins",
  "num_shells",
  "su_attempted",
  "is_guest_login",
  "srv_serror_rate",
  "diff_srv_rate",
  "num_outbound_cmds",
  "num_root",
  "num_access_files",
  "dst_host_count",
  "dst_host_rerror_rate",
  "dst_host_srv_serror_rate",
  "dst_host_serror_rate",
  "srv_count",
  "dst_bytes",
  "same_srv_rate",
  "dst_host_diff_srv_rate",
  "hot",
  "rerror_rate",
  "is_host_login",
  "dst_host_srv_diff_host_rate",
  "dst_host_same_src_port_rate",
  "srv_rerror_rate",
  "wrong_fragment",
  "flag_vec",
...

In [34]:
def oneHotKmeans(data: DataFrame,k: Int): Double ={

    val (protoEncoder,protoCol) = oneHotPipeline("protocol_type")
    val (serviceEncoder, serviceCol) = oneHotPipeline("service")
    val (flagEncoder, flagCol) = oneHotPipeline("flag")
    
    //Array of column names Array[Strings]
    val assembledCols = Set(data.columns: _*) --
            Seq("label","protocol_type","service","flag") ++
            Seq(protoCol, serviceCol,flagCol)
    
    val assembler = new VectorAssembler().setInputCols(assembledCols.toArray).setOutputCol("featureVector")

    //Method to scale features to be standard scores
    val scaler = new org.apache.spark.ml.feature.StandardScaler().
                    setWithMean(false).
                    setWithStd(true).
                    setInputCol("featureVector").
                    setOutputCol("scaledFeatureVector")
    //Model
    val kmeans = new org.apache.spark.ml.clustering.KMeans().
                    setSeed(Random.nextLong()).
                    setK(k).
                    setPredictionCol("cluster").
                    setFeaturesCol("scaledFeatureVector").
                    setMaxIter(40).
                    setTol(1.0e-5)
    //The pipelines can be composed into a single one by using the set stages
    val pipeline = new Pipeline().setStages(
                        Array(protoEncoder,serviceEncoder,flagEncoder,assembler,scaler,kmeans))
    
    val pipelineModel=pipeline.fit(data)
    
    //get the return result of the cost (being the total distance to assigned cluster center)
    val kmeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
    return kmeansModel.computeCost(pipelineModel.transform(data)) / data.count()
}

defined function oneHotKmeans

oneHotKmeans(data,30)

As identified earlier in my attempt to produce a xtab on the category assignments we are going to use a purity measure to quantify the homogeneity of the cluster with respect to the labels of the data. 

In [36]:
""" A version of the oneHotKMeans that returns the 
    pipelinemodel rather than the cost of the model"""

def fitOneHotKmeans(data: DataFrame,k: Int): org.apache.spark.ml.PipelineModel ={

    val (protoEncoder,protoCol) = oneHotPipeline("protocol_type")
    val (serviceEncoder, serviceCol) = oneHotPipeline("service")
    val (flagEncoder, flagCol) = oneHotPipeline("flag")
    
    val assembledCols = Set(data.columns: _*) --
            Seq("label","protocol_type","service","flag") ++
            Seq(protoCol, serviceCol,flagCol)
    
    val assembler = new VectorAssembler().setInputCols(assembledCols.toArray).setOutputCol("featureVector")

    //Method to scale features to be standard scores
    val scaler = new org.apache.spark.ml.feature.StandardScaler().
                    setWithMean(false).
                    setWithStd(true).
                    setInputCol("featureVector").
                    setOutputCol("scaledFeatureVector")
    //Model
    val kmeans = new org.apache.spark.ml.clustering.KMeans().
                    setSeed(Random.nextLong()).
                    setK(k).
                    setPredictionCol("cluster").
                    setFeaturesCol("scaledFeatureVector").
                    setMaxIter(40).
                    setTol(1.0e-5)
    val pipeline = new Pipeline().setStages(
                        Array(protoEncoder,serviceEncoder,flagEncoder,assembler,scaler,kmeans))
    val pipelineModel=pipeline.fit(data)
    //get the return result of the cost (being the total distance to assigned cluster center)
    
   return pipelineModel
}

res35_0: String = """ A version of the oneHotKMeans that returns the 
    pipelinemodel rather than the cost of the model"""
defined function fitOneHotKmeans

In [37]:
val pipelineModel = fitOneHotKmeans(data,180)

countByValue at StringIndexer.scala:140

8 / 8

countByValue at StringIndexer.scala:140

8 / 8

countByValue at StringIndexer.scala:140

...

collect at ClusteringSummary.scala:51

8 / 8

collect at ClusteringSummary.scala:51

8 / 8

pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_2809a6fc1d18

In [38]:
val clusterLabel = pipelineModel.transform(data).select("cluster","label").as[(Int,String)]

clusterLabel: Dataset[(Int, String)] = [cluster: int, label: string]

In [58]:
clusterLabel.describe().show()

describe at cmd57.sc:1

8 / 8

describe at cmd57.sc:1

1 / 1

+-------+------------------+------------+
|summary|           cluster|       label|
+-------+------------------+------------+
|  count|           4898431|     4898431|
|   mean|34.072824543205776|        null|
| stddev| 47.65101661554576|        null|
|    min|                 0|       back.|
|    max|               179|warezmaster.|
+-------+------------------+------------+



The table above shows that there is some spread in the categories. This is a quick and dirty way of first glipsing the data.

Try using cluster instead of computeCost to calculate the homogeneity of the clusters. First lests have a look at what kind of entropy numbers are present in the k=180 model

In [69]:
def entropy(counts: Iterable[Int]): Double = {
    val values = counts.filter(_ > 0)
    val n = values.map(_.toDouble).sum
        values.map { v =>
    val p = v / n
    -p * math.log(p)
    }.sum
}

defined function entropy

In [70]:
entropy(List(200000,1000))

res69: Double = 0.03134732940431287

In [71]:
"""
This is function returns the entropy of each cluster.
A measure of the 'model' entropy could be to take the weighted average of the entropies. However, other metrics might be suitable
in different circumstances. Eg. mean, mode, worst case. Or more specifically the distribution of normal labels across the clusters.
"""

val clusterEntropy = clusterLabel.
    groupByKey {case (cluster,_)=>cluster}.
    mapGroups {case (cluster,clusterLabels)=>
            val labels = clusterLabels.map{ case (cluster,label)=>label}.toSeq
            val labelCounts = labels.groupBy(identity).values.map(_.size)
        (cluster, entropy(labelCounts))
    }.toDF("cluster","entropy")


res70_0: String = """
This is function returns the entropy of each cluster.
A measure of the 'model' entropy could be to take the weighted average of the entropies. However, other metrics might be suitable
in different circumstances. Eg. mean, mode, worst case. Or more specifically the distribution of normal labels across the clusters.
"""
clusterEntropy: DataFrame = [cluster: int, entropy: double]

In [61]:
clusterEntropy.sort($"cluster").show()

show at cmd60.sc:1

8 / 8

show at cmd60.sc:1

8 / 8

+-------+--------------------+
|cluster|             entropy|
+-------+--------------------+
|      0|                 0.0|
|      1|                 0.0|
|      2|  0.0631455838094328|
|      3|0.001593718187644939|
|      4|  0.0432346908951394|
|      5| 0.06952076949182982|
|      6|0.036483642718501214|
|      7|  0.5839826991810138|
|      8| 0.11868333873597772|
|      9| 0.08596140779161125|
|     10|  0.1833795627037768|
|     11|0.009543190848675599|
|     12|  0.0897323332018887|
|     13|0.001278875015333...|
|     14| 0.06010234756185881|
|     15|   0.490211718340109|
|     16| 0.06304553779829346|
|     17| 0.18087270413184398|
|     18|  0.3854025364288664|
|     19|0.021095567156232536|
+-------+--------------------+
only showing top 20 rows



In [40]:
clusterLabel.take(10)

take at cmd39.sc:1

1 / 1

res39: Array[(Int, String)] = Array(
  (120, "normal."),
  (120, "normal."),
  (120, "normal."),
  (120, "normal."),
  (120, "normal."),
  (120, "normal."),
  (120, "normal."),
  (120, "normal."),
  (120, "normal."),
  (120, "normal.")
)

In [73]:
clusterLabel.createOrReplaceTempView("clusterData")
spark.sql("""
select cluster, cast(count(cluster) as int) as Counts from clusterData where label='normal.' group by cluster """).show()

show at cmd72.sc:2

8 / 8

show at cmd72.sc:2

1 / 1

show at cmd72.sc:2

4 / 4

+-------+------+
|cluster|Counts|
+-------+------+
|    171| 10041|
|     13| 41105|
|     38|  2041|
|     46|   846|
|     18|    72|
|    161|  4726|
|     39| 36430|
|      6|201742|
|    117|  2421|
|     86|  1085|
|    165|    24|
|    138|  2123|
|     51|    28|
|     94|    48|
|     17|     7|
|     74|   129|
|    121|     2|
|     62|     3|
|    141|     1|
|     23|     7|
+-------+------+
only showing top 20 rows



In [74]:
val normalClustering = spark.sql("""
select cluster,cast(count(cluster) as Int) as Counts from clusterData where label='normal.' group by cluster """)


normalClustering: DataFrame = [cluster: int, Counts: int]

Get the entropy of the classification of the transactions labelled 'normal.' Use collect.map(_.getInt(0)) to get the dataframe column into an iterable that can be passed to the entropy function

In [78]:
entropy(normalClustering.select("Counts").collect.map(_.getInt(0)))

collect at cmd77.sc:1

8 / 8

collect at cmd77.sc:1

8 / 8

res77: Double = 2.8175370886897375

In [65]:
val results = clusterLabel.select("cluster","label").
                groupBy("cluster","label").count().
                orderBy("cluster","label")
results.show()

            

show at cmd64.sc:4

8 / 8

show at cmd64.sc:4

8 / 8

+-------+----------+------+
|cluster|     label| count|
+-------+----------+------+
|      0|  neptune.|453714|
|      1|portsweep.|   117|
|      2|  neptune.|  1039|
|      2|portsweep.|     9|
|      2|    satan.|     2|
|      3|portsweep.|     2|
|      3|    satan.| 12190|
|      4|  neptune.|  1030|
|      4|portsweep.|     6|
|      4|    satan.|     1|
|      5|  neptune.|  1042|
|      5|portsweep.|     8|
|      5|    satan.|     4|
|      6|     back.|  1210|
|      6|   normal.|201742|
|      7|  ipsweep.|    15|
|      7|  neptune.|  1800|
|      7|   normal.|   483|
|      7|portsweep.|    10|
|      7|    satan.|     2|
+-------+----------+------+
only showing top 20 rows



results: Dataset[Row] = [cluster: int, label: string ... 1 more field]

Now use the model to make predictions

In [79]:
import org.apache.spark.ml.linalg.{Vector,Vectors}

import org.apache.spark.ml.linalg.{Vector,Vectors}

In [80]:
val kMeansModel = pipelineModel.stages.last.asInstanceOf[KMeansModel]
val centroids = kMeansModel.clusterCenters //will use in meansuring the squared distance to a centroid when assessing anomaliness

val clustered = pipelineModel.transform(data)




kMeansModel: KMeansModel = kmeans_0aae5cc7e4bd
centroids: Array[Vector] = Array(
  [0.0,0.0,0.0,2.058262486411676,0.0,0.10924430199690117,0.10837146929852963,1.1597990644273748,0.0,0.0,0.0,2.6183063736318073,0.0,0.0,0.0,0.0,0.0,0.0,2.615779838334647,0.7165692850778244,0.0,0.0,0.0,3.981159720732247,2.4622551959684405E-5,2.616585647985258,2.6186534641198334,0.046667971471668936,0.0,0.11770503907545483,0.607213036909946,0.0,3.5112225333396914E-6,0.0,0.0,4.0278446122257384E-5,4.78395098976446E-5,0.0,0.0,2.6167535806708884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.391919353595919,0.004802311454135209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.362700241244017E-4],
  [0.0,0.0,0.0,2.0582624864080343,2.0213616089940594,0.03887025679584396,0.03679854652946814,0.621257458271695

In [81]:
val threshold = clustered.select("cluster","scaledFeatureVector").as[(Int,Vector)].
                    map{ case (cluster,vec)=>Vectors.sqdist(centroids(cluster),vec)}.
                    orderBy($"value".desc).take(100).last


take at cmd80.sc:3

8 / 8

threshold: Double = 4658.554264626543

Note that the single output of the map is called value implicitly

Anomaly detection would then consist of determining a threshold distance and classifying as anomaly if greater than that threshold

Could do a test train split and then run the pipeline again to get the k=180 model then create confusion matrix to show the results of the classification. 

### Summary

Kmeans is one of several classification task models available. Kmeans is primarily an unsupervised method but in this case we did have labels and so the kmeans cluster could be used further down stream as an anomaly detector. However, the training of the model here did not use the labels. Instead, the computeCost method was used (total squared distance to centroids) and later the entropy of the classes was observed to see the level of purity (in the spirit of random forest's Gini and entropy of final class predictions.

Things of interest where the use of one hot encoded categorical variables. They are one hot encoded by instead of each one-hot encoded column being added to the data a StringIndexed column was used. This stores the vector of one hot encoded data and, apparently, stored meta-data related to the encoded that causes the data not be treated as independant numerical values as would be the case when adding 1 col per option. 

Feature normalization was also introduced with the org.apache.spark.ml.feature.StandardScaler class which converts distribution of each column to be a standard normal distribution including the vectorised categoricals (though its not clear if this is treated differently).

Each of the steps in transforming the data (one hot encoding, stringindexing, replacing columns, scaling, fitting model) call be assembled in a pipeline. Each step is just a transform in its own right.

Tuning (i.e. selecting k) is the one hyperparameter and it was set just with a simple grid search using the computeCost and (not shown) the model entropy (being the weighted sum of category entropies)
